# 4. Implementing Full-Text Search (FTS5)

In [ ]:
import json
import pandas as pd
from sqlite3 import connect

DB_PATH = '../sqlite-olt.db'

In [ ]:
with connect(DB_PATH) as db:
    comments_text_df = pd.read_sql("""
                    select 
                    json_extract(data, '$.objectID') as objectID,
                    json_extract(data, '$.comment_text') as comment_text,
                    length(json_extract(data, '$.comment_text')) as comment_text_length
                    from hn_items_raw
                    where json_extract(data, '$.comment_text') notnull
                    """,db
                    )

In [ ]:
comments_text_df

In [ ]:
with connect(DB_PATH) as db:
    db.execute("""
    DROP TABLE IF EXISTS comments_fts;
    """)
    db.execute("""
    CREATE VIRTUAL TABLE comments_fts USING fts5(objectID, comment_text);
    """)

In [ ]:
with connect(DB_PATH) as db:
    comments_text_df[['objectID', 'comment_text']].to_sql('comments_fts', db, if_exists='append', index=False)

In [ ]:
with connect(DB_PATH) as db:
    search_sqlite = pd.read_sql("""
                    select *
                    from comments_fts
                    where comment_text MATCH 'SQLite'
                    """,db
                    )
    
search_sqlite